# **RAG**
Retrieval Augmented Generation

## **1.환경준비**

### (1) 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB**

### (1) Chroma DB 구성

* DB 경로 지정
    * 없으면, 새로 폴더를 만들며 DB 생성
    * 있으면, 기존 DB 연결

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./data",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* DB 삭제는, 파이썬을 닫고 폴더를 삭제하면 됩니다.

### (2) INSERT

* 두가지 방법
    * (1) 단순 텍스트 입력 : 각 단위 텍스트를 리스트 형태로 입력 **.add_texts()**
    * (2) 텍스트와 메타데이터 입력 : 각 단위 텍스트와 메타정보를 함께 입력 **.add_documents()**

#### 1) .add_texts()

In [ ]:
input_list = ['test 데이터 입력1', 'test 데이터 입력2']

# 입력. 입력시 인덱스 저장(조회시 사용)
ind = database.add_texts(input_list)
ind

In [ ]:
database.get(ind)

#### 2) .add_documents()

In [ ]:
input_list2 = ['오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', '어제 주가는 큰 폭으로 상승했습니다.']
metadata = [{'category':'test'}, {'category':'test'}]

doc2 = [Document(page_content = input_list2[i], metadata = metadata[i]) for i in range(2)]
ind2 = database.add_documents(doc2)

In [ ]:
database.get(ind2)

### (2) 조회

#### 1) 전체 조회

In [ ]:
database.get()

#### 2) 조건조회
* 일반적인 조건 조회를 지원하지 않음.
    * 그래서 전체를 다 불러와서
    * dataframe으로 변환 후 조건 조회

In [ ]:
data = database.get()
data = pd.DataFrame(data)
data

In [ ]:
data.loc[data['metadatas'] == {'category': 'test'}]

#### 3) 유사 문서 조회

In [ ]:
# 문서 조회1
query = "오늘 낮 기온은?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시

In [ ]:
# 문서 조회2 : 유사도 점수도 함께 조회
query = "오늘 낮 기온은?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

### (3) 삭제

* 문서의 id 값으로 삭제합니다.

In [ ]:
data

In [ ]:
ids = ['3dcb6d22-8905-46d1-b1cf-875977616a47', '8dd69f36-edc8-4939-bb02-b05f289a049b']
database.delete(ids = ids)

In [ ]:
database.get()

## **3.데이터 벡터화**

### (1) DF to Vector DB

* 샘플데이터 : 오픈소스 생성형 AI에서 주의해야 할 10가지 사항
    * 기사를 csv로 만든 것입니다.

원문보기: https://www.ciokorea.com/news/337152#csidxc7d1d11066fad86a15937e4c3b29c6d



In [ ]:
data = pd.read_csv('sample.csv', encoding='utf-8')
data

In [ ]:
# Chroma 데이터베이스 인스턴스 생성
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)

In [ ]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환
text_list = data['내용'].tolist()

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content=text) for text in text_list]

print(documents)

In [ ]:
# Insert
database.add_documents(documents)

In [ ]:
database.get()

### (2) 질문으로 유사도 높은 문서 조회하기

In [ ]:
# 문서 조회 : 유사도 점수도 함께 조회
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"   # 질문할 문장
k = 5                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

### (3) 질문에 대한 답변 받기
* 절차
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 질문과 유사도높은 문서로 프롬프트 구성
    * GPT에 질문하고 답변 받기

* RetrievalQA
    * 랭체인에서 제공하는 체인 함수
    * RAG + QA

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
retriever = database.as_retriever()
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)

print(result["result"])

* 유사도 높은 문서 4개를 조회하는 것이 기본값임.

In [ ]:
result

* 유사도 높은 문서 k개로부터 프롬프트 구성하여 질의하기

In [ ]:
k = 5
retriever = database.as_retriever(search_kwargs={"k": k})

qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)
result